## Initialization 

In [ ]:
import numpy as np 
import pandas as pd 
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import torch.nn.functional as F
import numpy as np
import os
from PIL import Image 
from torch.utils.data import Dataset

# 
path = "/kaggle/input/ifood-data/" 
batch_size = 16
train_path = path + "train_set/train_set"
train_labels = path + "train_info.csv"
val_path = path + "val_set/val_set"
val_labels = path + "val_info.csv"
test_path = path + "test_set"

USE_GPU = True 
print_every = 11800

# hyperparameters used for training 
batch_size = 16
momentum = 0.9 
lr = 0.0005
epochs = 6

# initialize model weights and optimization algorithm 
model = models.resnet152(pretrained=True)
optimizer = optim.SGD(model.parameters(), 
                      lr=lr, momentum=momentum, nesterov=True)

save_path = "/kaggle/working/resnet152_10epochs.pth"
submission_name = "submission.txt"


## Loading Data

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. 
    Extendstorchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

class MyDataset(Dataset):
    """Custom dataset object for loading images and image labels
    Extends torch.utils.data.Dataset
    -----------
    image_path: location of the folder where the images are stored 
    image_names: list of all of the image files at the location 
    label_path: the path to the csv of the corresponding label to each image
    """                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    def __init__(self, 
                 image_path, 
                 image_names, 
                 label_path,  transform=None):

        self.image_path = image_path 
        self.image_names = image_names 
        self.transform = transform
        # reads a csv of all the labels 
        self.labels = pd.read_csv(label_path, 
                                  names=["img_name", "label"])
        
    def get_class_label(self, image_name):
        y = self.labels[self.labels["img_name"] == image_name].iloc[0]["label"]
        return y
        
    def __getitem__(self, index):
        path = self.image_path + "/" + self.image_names[index]
        # get the image and the label 
        x = Image.open(path)
        y = self.get_class_label(path.split('/')[-1])
        # apply transformations 
        if self.transform is not None:
            x = self.transform(x)
        return x, y
    
    def __len__(self):
        return len(self.image_names)


In [ ]:
val_names = [] 
# get the list of all images in the validation set  
for dirname, _, filenames in os.walk(val_path):
    for filename in filenames:
        val_names.append(filename)
        count += 1

train_names = [] 
# get the list of all images in the training set 
for dirname, _, filenames in os.walk(train_path):
    for filename in filenames:
        train_names.append(filename)
        count += 1

# the following will apply a list of transformations to the images
# transformations for the training set 
train_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.768986940,.6641706 ,0.5923363),
                         (0.18613161, 0.22524446, 0.23932885))])

# transformations for the validation set 
val_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.768986940,.6641706 ,0.5923363),
                         (0.18613161, 0.22524446, 0.23932885))])

# transformations for the test set 
test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.768986940,.6641706 ,0.5923363),
                         (0.18613161, 0.22524446, 0.23932885))])    

# create training and validation datasets 
train_data = MyDataset(train_path, 
                       train_names, 
                       train_labels, 
                       transform= train_transforms)  
val_data = MyDataset(val_path, 
                     val_names, 
                     val_labels, 
                     transform= val_transforms)

# load testing images from path 
test_data = ImageFolderWithPaths(test_path, 
                                 transform = test_transforms)


# create data loader objects for batch gradient descent 
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

# load an image one at a time for test set 
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1)


## GPU Usage

In [ ]:
!nvidia-smi

In [ ]:
# check if a GPU is available 
dtype = torch.float32 
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

## Model Training 

In [ ]:
def check_accuracy(loader, model):
    """calculates model accuracy on data loader 
    ----------
    loader: the dataset to check on, e.g train_loader, val_loader, test_loader
    model: a pytorch deeplearning model
    ----------
    output: prints the accuracy 
    """
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            # idenitfy the group with the highest value 
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

def train(model, optimizer, epochs):
    """performs gradient descent with optimizer 
    ------------
    model: a pytorch model 
    optimizer: optimizer from pytorch.optim 
    epochs: the number of epochs to train for 
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        
        for t, (x, y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % 100 == 0:
                print(t, loss.item())
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(val_loader, model)
                print()

In [ ]:
print(model)

In [ ]:
train(model, optimizer, epochs)

In [ ]:
check_accuracy(val_loader, model)

## Prediction 

In [ ]:
model.eval()
model.to(device=device)

# load testing images and write predicted labels into a txt file 
with open(submission_name, 'w') as file:
    file.write("img_name,label\n")
    with torch.no_grad():
        for x, y, path in test_loader:
            name = path[0][-15:]
            x = x.to(device=device, dtype=dtype)
            scores = model(x)
            out_labels = [int(x) for x in (torch.topk(scores, 3)[1][0])]
            file.write(name + "," + str(out_labels[0]) + " " + str(out_labels[1]) + " " + str(out_labels[2]) + "\n")
print("done writing")

In [ ]:
torch.save(model, save_path)